#  Text Processing Pipeline for Cal-Fresh Application Dataset

Author: Rocio Ng (DSWG Lead)

### Summary:  
* The purpose of this notebook is to prototype and test methods for processing free text entered into Applications for the CalFresh Program (https://www.getcalfresh.org/)
* Notebook applies helper modules that do the following:
    1. Apply light processing to text
    2. Attempt to correct mispelled words in the text
    3. Apply white-list to redact text that may contain personal information

### Resources:
* Peter Norvig's Spell Corrector Tutorial (http://norvig.com/spell-correct.html)
* Spanish Language Corpus - https://www.corpusdata.org/spanish.asp


## Load Libraries

In [1]:
import numpy as np
import pandas as pd
from langdetect import detect

import warnings
warnings.filterwarnings(action='once') # displays warnings only once

import os
import sys

# For loading Helper Functions
module_path = os.path.abspath(os.path.join('2-Helper-Modules'))
if module_path not in sys.path:
    sys.path.append(module_path)

# For Multicore processing
from multiprocessing import Pool

# Helper Modules
from spell_checking_functions_v3 import *
from text_processing_functions import *
from whitelist_functions import *

/Users/rociong/anaconda/envs/cal_env/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/rociong/Desktop/datasci-cfa-calfresh/2-Data-Processing/2-Helper-Modules/spell_checking_functions_v3.py:91: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/rociong/Desktop/datasci-cfa-calfresh/2-Data-Processing/2-Helper-Modules/../1-Text-Files/spanish-text.txt' mode='r' encoding='UTF-8'>
  self.WORDS = Counter(word_reader(open(path_to_spanish_text).read()))


the cat, is under the tree.


In [2]:
# Testing spell checker
en_spellchecker.correction_phrase("helpp meh with calfrsh whil i'm applying for ssi  .")

'help meh with catfish while i pm applying for ski.'

In [3]:
# print(detect('Hi')) # False Negative Results
# print(detect('I currently live in my truck'))
# print(detect("estoy embarazada"))

## Load data
* Make sure paths point to where data files are stored locally if you choose to rename/move things

In [4]:
text_df = pd.read_csv("../1-Data/500_sample_results.csv")
# text_df = pd.read_csv("1-Data-Files/orig_entRep_300.csv")

In [5]:
text_df.shape

(500, 2)

In [6]:
text_df.head()

,calfresh_app_id,with_entity_replacement
0,336864,i have blood cancer i have present in palomar ...
1,336863,I currently live in my truck
2,336860,"estoy embarazada, tengo CARDINAL semanas de em..."
3,336858,other then needing food as soon as possible i ...
4,336857,I am a single mother renting a room for CARDIN...


In [7]:
text_df = text_df.dropna(subset=['with_entity_replacement'])

In [8]:
text_df.shape

(497, 2)

## Processing

* Light text processing
* Count Spelling Errors
* Detect Langage

In [14]:
# demo 
initial_phrase_processing(" PERSON wenT to The Store at CARDINAL !!")

' PERSON went to the store at CARDINAL !!'

In [15]:
text_df['processed_phrase'] = text_df.with_entity_replacement\
    .apply(lambda x: initial_phrase_processing(x))

In [16]:
text_df.head()

,calfresh_app_id,with_entity_replacement,processed_phrase
0,336864,i have blood cancer i have present in palomar ...,i have blood cancer i have present in palomar ...
1,336863,I currently live in my truck,i currently live in my truck
2,336860,"estoy embarazada, tengo CARDINAL semanas de em...","estoy embarazada, tengo CARDINAL semanas de em..."
3,336858,other then needing food as soon as possible i ...,other then needing food as soon as possible i ...
4,336857,I am a single mother renting a room for CARDIN...,i am a single mother renting a room for CARDIN...


In [17]:
text_df['spelling_errors'] = text_df.processed_phrase
text_df = text_df.sort_values('spelling_errors', ascending = False)
text_df['language'] = text_df.processed_phrase.apply(lambda x: detect_B(x))

In [18]:
text_df.groupby(by = "language").count()

,calfresh_app_id,with_entity_replacement,processed_phrase,spelling_errors
language,,,,
en,484,484,484,484
es,13,13,13,13


In [19]:
# text_df[text_df.language.isin(['None'])]

In [20]:
text_df = text_df[~text_df.language.isin(['None'])]

## Apply Spell Checking Functions

* Convert Dataframe column of Phrases to List of Tuples (Word, Language) to enable Multiprocessing
* Run spell Correction_phrase function on text
* Append back to Dataframe

In [21]:
spelling_error_list = list(zip(text_df['processed_phrase'], text_df['language']))

In [22]:
# Preview
spelling_error_list[5:10]

[('yes im homeless', 'en'),
 ('yes i would appriciate the help that can be received thru this program thank you PERSON',
  'en'),
 ('working DATE just to eat free meal with shift. recycling to make bus money and hygiene needs met',
  'en'),
 ('will i be able to adjust information? i am moving out with roomates in DATE and was wondering ',
  'en'),
 ("we pay child support to person's ex wife. she doesn't pay any expenses for the kids but they live with her most of date. ",
  'en')]

In [ ]:
my_pool = Pool(processes=4) # change to number of cores in machine

In [ ]:
# For testing edge cases
spell_correction_language(("semesters", "en")) # will correct to a different word even though correct
# spell_correction_language(("alot", "en"))
# spell_correction_language(("farmacie", "es"))

In [ ]:
# Apply spell correction by language across all text
%time spelling_corrections = my_pool.map(spell_correction_language, spelling_error_list)

In [ ]:
spelling_corrections[5:10]

In [ ]:
# Append results to dataframe
text_df['spelling_corrections'] = spelling_corrections

In [ ]:
text_df.head()

In [ ]:
#subset_df = text_df.iloc[10:15]

In [ ]:
text_df.to_csv("gcf_circumstances_spell_correct.csv")

## Apply White List to Spell Corrected Phrases

In [ ]:
test_phrase = "This is a Test.   For Rocio. Hello. "
check_whitelist(test_phrase, whitelist_list, "replace")

In [ ]:
text_df['whitelisted_phrase'] = text_df.spelling_corrections\
    .apply(lambda x: check_whitelist(x, whitelist_list, "replace")[0])

In [ ]:
text_df.head()

In [ ]:
text_df.to_csv("gcf_circumstances_spell_correct_whitelist.csv")

## Validate Effectiveness of Corrections

In [ ]:
# text_df['words_removed_raw_words'] = text_df.original_additional_information_text\
#     .apply(lambda x: int(check_whitelist(x, whitelist_list)[1]))

# text_df['words_removed_spell_corrected'] = text_df.spelling_corrections\
#     .apply(lambda x: int(check_whitelist(x, whitelist_list, "remove")[1]))

# text_df = text_df\
#     .assign(pct_improvement = 100*(1 - (text_df.words_removed_spell_corrected/text_df.words_removed_raw_words)))\
#     .assign(improvement = text_df.words_removed_raw_words - text_df.words_removed_spell_corrected)

In [ ]:
# missing_words = ["test", "in", "an", "never", "work", "part", "house"]